# Crack Detection In Buildings

Crack detection has vital importance for structural health monitoring and inspection. We would like to train a network to detect Cracks, we will denote the images that contain cracks as positive and images with no cracks as negative.

To train the network, we will use a pre-trained model to classify between the positive and negetive samples. The particular **pre-trained model** will be **resnet18**.

## Download the data

Download the dataset and unzip the files in the data directory. (This may take some time)

In [17]:
#!wget --no-check-certificate https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip 

26550K .......... .......... .......... .......... .......... 94% 87.4K 18s
226600K .......... .......... .......... .......... .......... 94% 95.7K 18s
226650K .......... .......... .......... .......... .......... 94% 94.0K 18s
226700K .......... .......... .......... .......... .......... 94%  183K 18s
226750K .......... .......... .......... .......... .......... 94% 59.9K 18s
226800K .......... .......... .......... .......... .......... 94% 62.5K 18s
226850K .......... .......... .......... .......... .......... 94% 7.96M 18s
226900K .......... .......... .......... .......... .......... 94%  181K 18s
226950K .......... .......... .......... .......... .......... 94%  190K 18s
227000K .......... .......... .......... .......... .......... 94%  122K 18s
227050K .......... .......... .......... .......... .......... 94% 50.1M 17s
227100K .......... .......... .......... .......... .......... 94% 40.4M 17s
227150K .......... .......... .......... .......... .......... 94%  430K 17s


In [33]:
import zipfile
with zipfile.ZipFile("concrete_crack_images_for_classification.zip", "r") as z_ref:
    z_ref.extractall("concrete_crack_images")

We will install torchvision:

## Imports and Auxiliary Functions

The following are the libraries we are going to use for this lab. The torch.manual_seed() is for forcing the random function to give the same number every time we try to recompile it.



In [ ]:
import torchvision.models as models
from PIL import Image
import pandas as pd
from torchvision import transforms
import torch.nn as nn
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
torch.manual_seed(0)

In [ ]:
from matplotlib.pyplot import imshow
import time

## Dataset Class

To speed things up, we are going to use tensors instead of jpeg images. Therefore for each iteration, we will skip the reshape step, conversion step to tensors and normalization step.


In [ ]:
# Create your own dataset object
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/concrete_crack_images"
        positive="Positive"
        negative='Negative'

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in os.listdir(positive_file_path) if file.endswith(".pt")]
        negative_files=[os.path.join(negative_file_path,file) for file in os.listdir(negative_file_path) if file.endswith(".pt")]
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)     
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
               
        image=torch.load(self.all_files[idx])
        y=self.Y[idx]
                  
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y
    
print("done")

We create two dataset objects, one for the training data and one for the validation data.

In [ ]:
train_dataset = Dataset(train=True)
validation_dataset = Dataset(train=False)
print("done")